In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.a2c import A2C
from networks.a2c.actor import Actor
from networks.a2c.critic import Critic
from networks.a2c_lstm.actor import ActorLSTM
from networks.a2c_lstm.critic import CriticLSTM

import gym
import matplotlib.pyplot as plt
import torch

In [2]:
norm_cols = []
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 30
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-02-01',
        'end_date': '2021-03-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

actor_model = ActorLSTM(
    state_dim=batch_dur*3+3,
    action_dim=3,
)
critic_model = Critic(state_dim=batch_dur*3+3)
a2c = A2C(
    env=env, 
    actor=actor_model,
    critic=critic_model,
    n_actns=3,
    actor_optmz=torch.optim.Adam(actor_model.parameters(), lr=0.00001),
    critic_optmz=torch.optim.Adam(critic_model.parameters(), lr=0.00001),
    hyprprms={
        'gamma': 0.995,
    },
    log_freq=1,
    p_net_type='lstm',
    c_net_type='nn',
    load_models=False,
    mdl_pth='../models/a2c_al_cn',
)

In [ ]:
a2c.run(5000)


Ep: 0 | TS: 19970 | L: 1963277828.304 | R: 26.59 | P: -15.08 | R.Avg P: -15.08 | NW: 84.92 | R.Avg NW: 84.92 | R.U: 90
Ep: 1 | TS: 39940 | L: 1947149588.86 | R: 36.53 | P: 8.78 | R.Avg P: -3.15 | NW: 108.78 | R.Avg NW: 96.85 | R.U: 103
Ep: 2 | TS: 59910 | L: 1931103297.954 | R: 30.59 | P: -3.59 | R.Avg P: -3.3 | NW: 96.41 | R.Avg NW: 96.7 | R.U: 103
Ep: 3 | TS: 79880 | L: 1915140032.984 | R: 31.54 | P: -1.36 | R.Avg P: -2.81 | NW: 98.64 | R.Avg NW: 97.19 | R.U: 103
Ep: 4 | TS: 99850 | L: 1899263728.498 | R: 30.63 | P: -1.1 | R.Avg P: -2.47 | NW: 98.9 | R.Avg NW: 97.53 | R.U: 104
Ep: 5 | TS: 119820 | L: 1883472809.952 | R: 34.97 | P: 3.48 | R.Avg P: -1.48 | NW: 103.48 | R.Avg NW: 98.52 | R.U: 105
Ep: 6 | TS: 139790 | L: 1867765130.711 | R: 36.48 | P: 11.01 | R.Avg P: 0.31 | NW: 111.01 | R.Avg NW: 100.31 | R.U: 107
Ep: 7 | TS: 159760 | L: 1852144182.446 | R: 26.77 | P: -15.54 | R.Avg P: -1.67 | NW: 84.46 | R.Avg NW: 98.32 | R.U: 105
Ep: 8 | TS: 179730 | L: 1836611934.718 | R: 45.28 | P:

In [ ]:
a2c.evaluate(start_dt='2021-01-02 07:00', duration=30)

## Visualizations

### Training 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

### Evaluation

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.eval_logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# torch.save(a2c.actor.state_dict(), '../models/a2c_cl_an/actor')
# torch.save(a2c.critic.state_dict(), '../models/a2c_cl_an/critic')

In [ ]:
import pickle
from datetime import datetime

temp = dict(a2c.logs)
with open(f"../pickles/a2c_logs_{datetime.now()}.pickle","wb") as f:
    pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)

### Observations

- A2C performs well when actor is chained with LSTM and critic with Neural Network
- If the actor is chained with Neural Network we dont seem to learn the task